# Tremor analysis

This tutorial shows how to run the tremor pipeline on prepared IMU or gyroscope data to obtain weekly aggregated tremor measures.

## Load example data

Example IMU data (8 minutes) from a participant of the Personalized Parkinson Project is loaded. The data was prepared as explained in data_preparation.ipynb. The prepared data contains both accelerometer and gyroscope data, but only gyroscope data is necessary for running the tremor pipeline.

In [1]:
import os
from paradigma.util import load_tsdf_dataframe

path_to_data =  '../../tests/data'
path_to_prepared_data = os.path.join(path_to_data, '1.prepared_data', 'imu')
df_data, metadata_time, metadata_values = load_tsdf_dataframe(path_to_prepared_data, prefix='IMU')

df_data

,time,accelerometer_x,accelerometer_y,accelerometer_z,gyroscope_x,gyroscope_y,gyroscope_z
0,0.00000,0.550718,0.574163,-0.273684,-115.670732,32.012195,-26.097561
1,0.01004,0.535885,0.623445,-0.254545,-110.609757,34.634146,-24.695122
2,0.02008,0.504306,0.651675,-0.251675,-103.231708,36.768293,-22.926829
3,0.03012,0.488517,0.686603,-0.265550,-96.280488,38.719512,-21.158537
4,0.04016,0.494258,0.725359,-0.278469,-92.560976,41.280488,-20.304878
...,...,...,...,...,...,...,...
72942,730.74468,0.234928,-0.516268,-0.802871,0.975610,-2.256098,2.256098
72943,730.75472,0.245455,-0.514354,-0.806699,0.304878,-1.707317,1.768293
72944,730.76476,0.243541,-0.511005,-0.807177,0.304878,-1.585366,1.890244
72945,730.77480,0.240191,-0.514354,-0.808134,0.000000,-1.280488,1.585366


## Step 1: Preprocess data

 The gyroscope data is resampled at the frequency specified in config (default 100 Hz). The accelerometer data is removed from the dataframe.

In [2]:
from paradigma.config import IMUConfig
from paradigma.preprocessing import preprocess_imu_data

config = IMUConfig()
df_preprocessed_data = preprocess_imu_data(df_data, config, sensor='gyroscope')

df_preprocessed_data

,time,gyroscope_x,gyroscope_y,gyroscope_z
0,0.00,-115.670732,32.012195,-26.097561
1,0.01,-110.636301,34.624710,-24.701537
2,0.02,-103.292766,36.753000,-22.942002
3,0.03,-96.349062,38.692931,-21.175227
4,0.04,-92.585735,41.237328,-20.311531
...,...,...,...,...
73074,730.74,1.150220,-2.561552,2.440945
73075,730.75,0.588721,-1.917765,1.948620
73076,730.76,0.270257,-1.626831,1.813725
73077,730.77,0.185022,-1.451942,1.793145


## Step 2: Extract tremor features

The preprocessed gyroscope data is windowed (default window lenght of 4 seconds) and for each window the tremor feautures are extracted. The features consist of 12 mel-frequency cepstral coefficients (MFCCs), the frequency of the peak in the estimated power spectral density, the low frequency power and the tremor power.

In [3]:
from paradigma.config import TremorFeatureExtractionConfig
from paradigma.tremor.tremor_analysis import extract_tremor_features

config = TremorFeatureExtractionConfig()
df_features = extract_tremor_features(df_preprocessed_data, config)

df_features

,time,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,freq_peak,low_freq_power,tremor_power
0,0.0,8.583226,4.163693,0.716525,0.865086,0.163955,0.032869,0.272959,0.248040,-0.299861,-0.018083,-0.100187,0.161498,1.00,13277.614531,1020.094627
1,4.0,4.900216,4.244692,0.666047,0.843129,-0.259316,0.269170,-0.089431,0.382559,0.136254,-0.039932,-0.038591,-0.059728,1.25,1310.703842,87.538668
2,8.0,2.544072,3.960958,1.101140,0.259860,0.066694,-0.339588,-0.107608,0.224469,0.001854,0.165404,0.069638,0.129111,3.50,215.567337,56.805816
3,12.0,6.542247,4.968952,1.004634,0.707018,-0.162786,0.005732,0.224887,0.152696,0.153201,0.141679,-0.104205,0.248928,1.00,7808.868582,566.219495
4,16.0,8.306986,2.596011,0.791752,0.399327,0.237048,0.570531,0.459708,0.317857,-0.104083,0.061507,-0.029085,0.130021,1.00,3945.127377,174.899788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,708.0,-3.025538,3.060808,1.096377,0.731820,-0.205085,0.275844,0.088779,-0.133286,-0.186760,-0.106265,-0.108878,0.212462,2.50,4.952011,0.397532
178,712.0,3.599376,3.480070,0.570259,0.709622,0.191344,0.515849,0.228787,0.199043,0.179561,0.120063,0.246163,0.300809,1.00,570.735779,32.044727
179,716.0,-1.852881,3.040926,0.653718,-0.114224,-0.157576,0.195950,0.060589,0.271852,-0.128228,0.009769,-0.052406,0.129333,1.00,3.990732,1.261435
180,720.0,-2.625325,3.290699,0.324278,0.315394,-0.144384,-0.038387,0.396184,0.044741,-0.144045,-0.235692,0.006585,0.188508,2.50,3.638188,1.217930


## Step 3: Detect tremor

A pretrained logistic regression classifier is used to predict the tremor probability (pred_tremor_proba) of each window, based on the MFCCs. Using the prespecified threshold, a tremor label of 0 (no tremor) or 1 (tremor) is assigned (pred_tremor_logreg). Next, two extra criteria for rest tremor are applied: the frequency of the peak should be between 3-7 Hz and the low frequency power should be below a prespecified threshold. The final tremor label is saved in pred_tremor_checked.

In [4]:
from paradigma.config import TremorDetectionConfig
from paradigma.tremor.tremor_analysis import detect_tremor

config = TremorDetectionConfig()
path_to_classifier_input = os.path.join(path_to_data, '0.classification', 'tremor')
df_predictions = detect_tremor(df_features, config, path_to_classifier_input)

df_predictions

,time,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,freq_peak,low_freq_power,tremor_power,pred_tremor_proba,pred_tremor_logreg,pred_tremor_checked
0,0.0,8.583226,4.163693,0.716525,0.865086,0.163955,0.032869,0.272959,0.248040,-0.299861,-0.018083,-0.100187,0.161498,1.00,13277.614531,1020.094627,0.000916,0,0
1,4.0,4.900216,4.244692,0.666047,0.843129,-0.259316,0.269170,-0.089431,0.382559,0.136254,-0.039932,-0.038591,-0.059728,1.25,1310.703842,87.538668,0.002644,0,0
2,8.0,2.544072,3.960958,1.101140,0.259860,0.066694,-0.339588,-0.107608,0.224469,0.001854,0.165404,0.069638,0.129111,3.50,215.567337,56.805816,0.001783,0,0
3,12.0,6.542247,4.968952,1.004634,0.707018,-0.162786,0.005732,0.224887,0.152696,0.153201,0.141679,-0.104205,0.248928,1.00,7808.868582,566.219495,0.002999,0,0
4,16.0,8.306986,2.596011,0.791752,0.399327,0.237048,0.570531,0.459708,0.317857,-0.104083,0.061507,-0.029085,0.130021,1.00,3945.127377,174.899788,0.000947,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,708.0,-3.025538,3.060808,1.096377,0.731820,-0.205085,0.275844,0.088779,-0.133286,-0.186760,-0.106265,-0.108878,0.212462,2.50,4.952011,0.397532,0.000187,0,0
178,712.0,3.599376,3.480070,0.570259,0.709622,0.191344,0.515849,0.228787,0.199043,0.179561,0.120063,0.246163,0.300809,1.00,570.735779,32.044727,0.001276,0,0
179,716.0,-1.852881,3.040926,0.653718,-0.114224,-0.157576,0.195950,0.060589,0.271852,-0.128228,0.009769,-0.052406,0.129333,1.00,3.990732,1.261435,0.001945,0,0
180,720.0,-2.625325,3.290699,0.324278,0.315394,-0.144384,-0.038387,0.396184,0.044741,-0.144045,-0.235692,0.006585,0.188508,2.50,3.638188,1.217930,0.001606,0,0


## Step 4: Quantify tremor

The final step is to quantify the amount of tremor time and tremor power, aggregated over all windows in the input dataframe.
Tremor time is calculated as the number of the detected tremor windows, as percentage of the number of windows 
while the arm is at rest or in stable posture (when the low frequency power is below a specified threshold). For tremor power the following aggregates are derived: the median, mode and percentile of tremor power specified in the configuration object (default is 90th percentile). The aggregated tremor measures and metadata are stored in a json file.

In [5]:
from paradigma.config import TremorQuantificationConfig
from paradigma.tremor.tremor_analysis import quantify_tremor

config = TremorQuantificationConfig()
d_tremor_aggregates = quantify_tremor(df_predictions, config)

d_tremor_aggregates

{'metadata': {'nr_windows_total': 182, 'nr_windows_rest': 89},
 'weekly_tremor_measures': {'tremor_time': 11.235955056179774,
  'tremor_power_median': 0.5451560244011688,
  'tremor_power_mode': 0.52,
  'tremor_power_90th_perc': 1.796145868028196}}